# IMPORTS

In [2]:
import pandas as pd

import my_pickle as mp
import my_split as ms
import my_dataframes as md

from importlib import reload

import warnings
warnings.filterwarnings('ignore')

In [52]:
reload(md)

<module 'my_dataframes' from '/Users/gandalf/Documents/Galvanize/MatchingService/my_dataframes.py'>

# READ IN

In [73]:
# message_df = md.get_message_df()
convo_df = md.get_conversation_df(message_df)
# user_df = md.get_user_df()

created conversation dataframe


In [87]:
def my_to_datetime_2(x):
    if isinstance(x, int): return pd.to_datetime(x*1000000)
    else: return None

def my_to_timedelta_2(x):
    if isinstance(x, int):return pd.to_timedelta(x).days
    else: return None

In [88]:
convo_df['timestampd'] = convo_df.timestamp.apply(my_to_datetime_2)
convo_df['timestampd_receiver'] = convo_df.timestamp_receiver.apply(my_to_datetime_2)
convo_df['timed_to_respond'] = convo_df.timestamp_receiver.apply(my_to_timedelta_2)
convo_df[['timestampd','timestampd_receiver','timed_to_respond']]

,timestampd,timestampd_receiver,timed_to_respond
24,2017-11-12 13:27:23.250,NaT,NaN
25,2017-09-11 02:41:19.188,NaT,NaN
26,2017-10-01 19:01:24.198,NaT,NaN
27,2017-09-06 20:03:30.419,NaT,NaN
28,2017-09-06 20:04:10.818,NaT,NaN
29,2017-07-03 20:14:06.271,NaT,NaN
30,2017-07-03 20:18:55.419,NaT,NaN
31,2017-07-03 20:23:06.571,NaT,NaN
32,2017-07-03 20:12:04.855,NaT,NaN
33,2017-09-06 04:52:29.649,NaT,NaN


# ECT SPLIT

In [18]:
# find the split!
cutoff = ms.ect_find_split(convo_df, .7)
print("split on:                      {}".format(pd.to_datetime(cutoff*1000000)))

# do the split!
safe_convo_df = ms.ect_make_split(convo_df, cutoff)
print("most recent convo data:        {}".format(pd.to_datetime(safe_convo_df.timestamp.max()*1000000)))

safe_message_df = ms.ect_make_split(message_df, cutoff)
print("most recent message data:      {}".format(pd.to_datetime(safe_message_df.timestamp.max()*1000000)))

# safe_lastmessage_df = ms.ect_make_split(last_df, cutoff)
# print("most recent last message data: {}".format(pd.to_datetime(safe_lastmessage_df.timestamp.max()*1000000)))

split on:                      2017-10-01 06:37:20.466000
most recent convo data:        2017-10-01 06:37:20.466000
most recent message data:      2017-10-01 06:37:20.466000


# REMOVE ROWS FROM CONVOS WITH MISSING UIDS (not now)

In [ ]:
# safe_convo_df = md.remove_bad_uids(safe_convo_df, user_df)  

# PICKLE EVERYTHING!

https://www.youtube.com/watch?v=yYey8ntlK_E

In [19]:
mp.json_it(user_df,'data_user')
mp.json_it(safe_message_df,'data_message')  
mp.json_it(safe_convo_df,'data_convo')  
# mp.json_it(lastmessage_df,'lastmessage_df')

In [20]:
safe_convo_df.head()

,response,uid_sender,len_sender,len_receiver,mid_sender,mid_receiver,timestamp,convo_length,uid_receiver
conversation_id,,,,,,,,,
02sjTzcsaO02omVxM8Xm,False,02omVxM8Xm,21,None,-Ktio--3zmDFQg8sB6yL,None,1505097679188,1,02sjTzcsaO
07lVtClAEQ01iDVeaXl1,False,01iDVeaXl1,1,None,-KtNm_grgv_VgpuPaKK2,None,1504728210419,1,07lVtClAEQ
0AFU6guKDt01iDVeaXl1,False,01iDVeaXl1,1,None,-KtNmjZOHhaqIiSvzJVT,None,1504728250818,1,0AFU6guKDt
0AFU6guKDt01mfWG4vUm,False,0AFU6guKDt,4,None,-Ko94f8tAmscNPu6Jgat,None,1499112846271,1,01mfWG4vUm
0AFU6guKDt06y5iMXoCP,False,0AFU6guKDt,1,None,-Ko95lnJtMsNzXmjqWKA,None,1499113135419,1,06y5iMXoCP
